Get the new articles of 20 minuten
using their rss feed.
Articles are considered as new if published after the last check date


In [116]:
import pandas as pd
import numpy as np
import feedparser
import datetime
import pathlib

In [117]:
# Get feed
url = 'https://partner-feeds.20min.ch/rss/20minuten/zentralschweiz'
feed = feedparser.parse(url)
print(f"Articles in feed : {len(feed.entries)}")

Articles in feed : 31


In [118]:
# Get last check date from global last_check_articles_20min
# liverss_and_coms_path = pathlib.Path.home() / 'Desktop' / 'SocialMediaMonitor' / 'data' / 'raw' / 'liverss_and_coms_20min.csv'
liverss_and_coms_path = '~/Desktop/SocialMediaMonitor/data/raw/liverss_and_coms_20min.csv'
liverss_and_coms_df = pd.read_csv(liverss_and_coms_path)
last_check_articles_20min = pd.to_datetime(liverss_and_coms_df['published'].max())
last_check_articles_20min

Timestamp('2022-05-10 02:53:40+0000', tz='UTC')

In [111]:
# Computing the newly aquired articles -> not used up to now (04.05.2022)
articles_to_keep = []
for entry in feed.entries:
    entry_date = pd.to_datetime(entry.published)
    # print(f"Entry date : {entry_date}")
    if last_check_articles_20min < entry_date:
        articles_to_keep.append(entry)
print(f"New articles in feed since last check : {len(articles_to_keep)}")

New articles in feed since last check : 30


In [112]:
# Compute temporary Pandas dataframe to further process beneath
liverss_tmp_df = pd.DataFrame.from_dict(articles_to_keep, orient='columns')
# liverss_tmp_df['published'] = pd.to_datetime(liverss_tmp_df['published'])
liverss_tmp_df.head()

,title,title_detail,links,link,summary,summary_detail,id,guidislink,published,published_parsed
0,«Polarstar»-Besitzer sucht Nachfolger für Revi...,"{'type': 'text/plain', 'language': None, 'base...","[{'rel': 'alternate', 'type': 'text/html', 'hr...",https://www.20min.ch/730773678727,"Schaumpartys, Fasnachtsbälle, viel Alkohol und...","{'type': 'text/html', 'language': None, 'base'...",https://www.20min.ch/story/polarstar-besitzer-...,False,"Mon, 09 May 2022 17:09:30 GMT","(2022, 5, 9, 17, 9, 30, 0, 129, 0)"
1,Bei Enkeltrick-Betrügern klickten die Handsche...,"{'type': 'text/plain', 'language': None, 'base...","[{'rel': 'alternate', 'type': 'text/html', 'hr...",https://www.20min.ch/577039134851,Bei einer geplanten Geldübergabe konnten zivil...,"{'type': 'text/html', 'language': None, 'base'...",https://www.20min.ch/story/bei-enkeltrick-betr...,False,"Mon, 09 May 2022 15:44:54 GMT","(2022, 5, 9, 15, 44, 54, 0, 129, 0)"
2,Servette-Imeri fliegt innert einer Minute mit ...,"{'type': 'text/plain', 'language': None, 'base...","[{'rel': 'alternate', 'type': 'text/html', 'hr...",https://www.20min.ch/811526744044,Der FC Luzern darf sich wieder Hoffnungen auf ...,"{'type': 'text/html', 'language': None, 'base'...",https://www.20min.ch/video/servette-imeri-flie...,False,"Sun, 08 May 2022 17:31:24 GMT","(2022, 5, 8, 17, 31, 24, 6, 128, 0)"
3,In steilem Gelände – Ortsunkundiger Fahrer ver...,"{'type': 'text/plain', 'language': None, 'base...","[{'rel': 'alternate', 'type': 'text/html', 'hr...",https://www.20min.ch/897536422260,"Verbrechen, Unfälle und Fahndungen: Die neuest...","{'type': 'text/html', 'language': None, 'base'...",https://www.20min.ch/story/polizei-deckt-gross...,False,"Sun, 08 May 2022 10:17:25 GMT","(2022, 5, 8, 10, 17, 25, 6, 128, 0)"
4,Stadtfest will auf Bargeld verzichten – und lö...,"{'type': 'text/plain', 'language': None, 'base...","[{'rel': 'alternate', 'type': 'text/html', 'hr...",https://www.20min.ch/402438287701,Das Stadtfest soll dieses Jahr bargeldlos stat...,"{'type': 'text/html', 'language': None, 'base'...",https://www.20min.ch/story/stadtfest-will-auf-...,False,"Sat, 07 May 2022 16:22:44 GMT","(2022, 5, 7, 16, 22, 44, 5, 127, 0)"


In [113]:
# Cleaning created dataframe of 20min liverss_tmp_df
liverss_df = pd.DataFrame()
liverss_df['title'] = liverss_tmp_df['title']
liverss_df['summary'] = liverss_tmp_df['summary']
liverss_df['author'] = ''
liverss_df['is_com_or_sub'] = 'sub'
liverss_df['link'] = liverss_tmp_df['link']
liverss_df['id'] = liverss_tmp_df['id'].str[-12:]
liverss_df['published'] = liverss_tmp_df['published']
liverss_df['published_parsed'] = liverss_tmp_df['published_parsed']
liverss_df.head()


,title,summary,author,is_com_or_sub,link,id,published,published_parsed
0,«Polarstar»-Besitzer sucht Nachfolger für Revi...,"Schaumpartys, Fasnachtsbälle, viel Alkohol und...",,sub,https://www.20min.ch/730773678727,730773678727,"Mon, 09 May 2022 17:09:30 GMT","(2022, 5, 9, 17, 9, 30, 0, 129, 0)"
1,Bei Enkeltrick-Betrügern klickten die Handsche...,Bei einer geplanten Geldübergabe konnten zivil...,,sub,https://www.20min.ch/577039134851,577039134851,"Mon, 09 May 2022 15:44:54 GMT","(2022, 5, 9, 15, 44, 54, 0, 129, 0)"
2,Servette-Imeri fliegt innert einer Minute mit ...,Der FC Luzern darf sich wieder Hoffnungen auf ...,,sub,https://www.20min.ch/811526744044,811526744044,"Sun, 08 May 2022 17:31:24 GMT","(2022, 5, 8, 17, 31, 24, 6, 128, 0)"
3,In steilem Gelände – Ortsunkundiger Fahrer ver...,"Verbrechen, Unfälle und Fahndungen: Die neuest...",,sub,https://www.20min.ch/897536422260,897536422260,"Sun, 08 May 2022 10:17:25 GMT","(2022, 5, 8, 10, 17, 25, 6, 128, 0)"
4,Stadtfest will auf Bargeld verzichten – und lö...,Das Stadtfest soll dieses Jahr bargeldlos stat...,,sub,https://www.20min.ch/402438287701,402438287701,"Sat, 07 May 2022 16:22:44 GMT","(2022, 5, 7, 16, 22, 44, 5, 127, 0)"


In [115]:
# Saving generated data to raw data folder
liverss_df.to_csv(r'~/Desktop/SocialMediaMonitor/data/raw/liverss_20Min.csv', index = False)